In [1]:
from shapely import geometry
import geopandas as gpd
from srai.constants import WGS84_CRS, REGIONS_INDEX
from srai.loaders.osm_loaders import OSMTileLoader
from srai.regionizers import H3Regionizer, SlippyMapRegionizer
from srai.joiners import IntersectionJoiner
from srai.embedders import CountEmbedder
from srai.utils import geocode_to_region_gdf
from srai.plotting.folium_wrapper import plot_regions

/home/marcin/Documents/mgr/nw/nw_venv/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/tmp/ipykernel_17325/74256059.py:2: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/

In [2]:
city_gdf = geocode_to_region_gdf("Wroclaw")
city_gdf

,geometry
region_id,
"Wrocław, Lower Silesian Voivodeship, Poland","POLYGON ((16.80734 51.13895, 16.80859 51.13887..."


In [18]:
regionizer = SlippyMapRegionizer(zoom=12)
regions_gdf = regionizer.transform(city_gdf)
folium_map = city_gdf.explore(tiles="CartoDB positron")
print(regions_gdf.shape[0])
plot_regions(regions_gdf, map=folium_map)

18


In [19]:
import os
from tqdm import tqdm
n_tiles = []
with open('data/cities.txt', 'r') as cities_file:
    for city in tqdm(cities_file):
        print(city)
        city = city.replace("\n", "")
        path = f"data/tiles/{city}"
        os.mkdir(path)
        loader = OSMTileLoader(tile_server_url="https://tile.openstreetmap.de/", zoom=12, data_collector="save", storage_path=path)
        city_gdf = geocode_to_region_gdf(city)
        loader.load(city_gdf)

0it [00:00, ?it/s]

Wroclaw



1it [00:11, 11.83s/it]


In [5]:
import os
from tqdm import tqdm
n_tiles = []
with open('data/cities_v2.txt', 'r') as cities_file:
    for city in tqdm(cities_file, total=56):
        city = city.replace("\n", "")
        loader = SlippyMapRegionizer(z=16)
        n_tiles.append({'city': city, "data": loader.transform(geocode_to_region_gdf(city)).shape[0]})

 50%|█████     | 56/112 [02:24<02:24,  2.58s/it]


In [6]:
import pandas as pd

df = pd.DataFrame(n_tiles)
df.sum()

city    Tirana, AlbaniaYerevan, ArmeniaVienna, Austria...
data                                               259466
dtype: object

In [7]:
pd.set_option('display.max_rows', df.shape[0]+1)
display(df.sort_values(by="data", ascending=False))

,city,data
44,"Ankara, Turkey",118002
54,"Moscow, Russia",9243
12,"Helsinki, Finland",7982
15,"Berlin, Germany",6786
19,"Rome, Italy",6631
47,"New York City, USA",5908
49,"Los Angeles, USA",5880
45,"Kyiv, Ukraine",5807
55,"Nur-Sultan, Kazakhstan",5772
34,"Gdańsk, PL",5655
